In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
# df10 = pd.read_csv("ATP_Matches/atp_matches_2010.csv")
# df11 = pd.read_csv("ATP_Matches/atp_matches_2011.csv")

# dfs = {}

# # Loop through years from 2010 to 2023
# for year in range(2010, 2024):
#     filename = f"ATP_Matches/atp_matches_{year}.csv"
#     dfs[f'df{year}'] = pd.read_csv(filename)


# Initialize an empty list to store DataFrames
dfs = []

# Loop through years from 2010 to 2023
for year in range(2010, 2024):
    filename = f"ATP_Matches/atp_matches_{year}.csv"
    df = pd.read_csv(filename)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Display the merged DataFrame
print(merged_df)


                             tourney_id                  tourney_name surface  \
0                              2010-339                      Brisbane    Hard   
1                              2010-339                      Brisbane    Hard   
2                              2010-339                      Brisbane    Hard   
3                              2010-339                      Brisbane    Hard   
4                              2010-339                      Brisbane    Hard   
...                                 ...                           ...     ...   
39490  2023-M-DC-2023-WG2-PO-RSA-LUX-01  Davis Cup WG2 PO: RSA vs LUX     NaN   
39491  2023-M-DC-2023-WG2-PO-TUN-CYP-01  Davis Cup WG2 PO: TUN vs CYP     NaN   
39492  2023-M-DC-2023-WG2-PO-TUN-CYP-01  Davis Cup WG2 PO: TUN vs CYP     NaN   
39493  2023-M-DC-2023-WG2-PO-TUN-CYP-01  Davis Cup WG2 PO: TUN vs CYP     NaN   
39494  2023-M-DC-2023-WG2-PO-TUN-CYP-01  Davis Cup WG2 PO: TUN vs CYP     NaN   

       draw_size tourney_le

In [8]:
df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [9]:
importantData = df[['tourney_name', 'surface', 'tourney_level', 
                    'winner_name', 'winner_age', 
                    'loser_name', 'loser_age', 
                    'score', ]]
importantData = importantData.dropna()
importantData

,tourney_name,surface,tourney_level,winner_name,winner_age,loser_name,loser_age,score
0,United Cup,Hard,A,Taylor Fritz,25.1,Matteo Berrettini,26.7,7-6(4) 7-6(6)
1,United Cup,Hard,A,Frances Tiafoe,24.9,Lorenzo Musetti,20.8,6-2 0-0 RET
2,United Cup,Hard,A,Taylor Fritz,25.1,Hubert Hurkacz,25.8,7-6(5) 7-6(5)
3,United Cup,Hard,A,Frances Tiafoe,24.9,Kacper Zuk,23.9,6-3 6-3
4,United Cup,Hard,A,Stefanos Tsitsipas,24.3,Matteo Berrettini,26.7,4-6 7-6(2) 6-4
...,...,...,...,...,...,...,...,...
2951,Davis Cup WG2 R1: SLO vs LUX,Clay,D,Blaz Rola,32.9,Alex Knaff,25.7,6-4 6-2
2952,Davis Cup WG2 R1: SLO vs LUX,Clay,D,Chris Rodesch,22.1,Bor Artnak,19.2,6-2 7-6(3)
2953,Davis Cup WG2 R1: URU vs EGY,Clay,D,Mohamed Safwat,32.9,Ignacio Carou,24.1,7-5 6-3
2954,Davis Cup WG2 R1: URU vs EGY,Clay,D,Karim Mohamed Maamoun,32.4,Franco Roncadelli,23.5,6-1 4-6 7-5


In [6]:
X = importantData[['tourney_name', 'surface', 'tourney_level']]
Y = importantData[['winner_name']]

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [23]:
logModel = LogisticRegression()
logModel.fit(X_train, Y_train)

ValueError: could not convert string to float: 'South Orange'